# 前言：
mysql的重点梳理。

# 1. Navicat

用于连接数据库
- 查找
- 修改
- 更新
...

将操作转换为SQL语句。

# 2. 导出数据库数据

- 导出现有数据库数据：
    - mysqldump -u用户名 -p密码 数据库名称 >导出文件路径           # 结构+数据
    - mysqldump -u用户名 -p密码 -d 数据库名称 >导出文件路径       # 结构 
- 导入现有数据库数据：
    - mysqldump -uroot -p密码  数据库名称 < 文件路径  

# 3. 练习题

### 1. 自行创建测试数据

自行创建或mysqldump 导入数据和结构。

### 2. 查询“生物”课程比“物理”课程成绩高的所有学生的学号

思路：
- 获取所有有生物课程的人（学号，成绩） - 临时表
- 获取所有有物理课程的人（学号，成绩） - 临时表
- 根据【学号】连接两个临时表：
        学号  物理成绩   生物成绩
 
- 然后再进行筛选

### 3. 查询平均成绩大于60分的同学的学号和平均成绩

### 4. 查询所有同学的学号、姓名、选课数、总成绩；

### 5. 查询姓“李”的老师的个数；

### 6. 查询没学过“李平”老师课的同学的学号、姓名；

- 先查到“李平老师”老师教的所有课ID
- 获取选过课的所有学生ID
- 学生表中筛选

select * from student where sid not in (
        select DISTINCT student_id from score where score.course_id in (
            select cid from course left join teacher on course.teacher_id = teacher.tid where tname = '李平老师'
        )
    )

### 7. 查询学过“001”并且也学过编号“002”课程的同学的学号、姓名；

思路：
- 先查到既选择001又选择002课程的所有同学
- 根据学生进行分组，如果学生数量等于2表示，两门均已选择

select student.sid, student.sname from student left join (
	select student_id,count(1) as C from 
		(select student_id,course_id from score where course_id in (1,2)) as A
	group by student_id having C=2) as B on student.sid = B.student_id;

从性能上讲，下面这个sql语句更好，通过left join减少了一次字查询。

### 8. 查询学过“李平”老师所教的所有课的同学的学号、姓名；

同上，只不过将001和002变成 in (叶平老师的所有课)

left join减少子查询。

### 9. 查询课程编号“002”的成绩比课程编号“001”课程低的所有同学的学号、姓名；

### 10. 查询有课程成绩小于60分的同学的学号、姓名；

select sid,sname from student where sid in (
        select distinct student_id from score where num < 60
    )

### 11. 查询没有学全所有课的同学的学号、姓名；

### 12. 查询至少有一门课与学号为“001”的同学所学相同的同学的学号和姓名；

思路：
- 获取 001 同学选择的所有课程
- 获取课程在其中的所有人以及所有课程
- 根据学生筛选，获取所有学生信息
- 再与学生表连接，获取姓名

### 13. 查询学过学号为“001”同学所有课的其他同学学号和姓名；

- 先找到和001的学过的所有人
- 然后个数 ＝ 001所有学科     ＝＝> 其他人可能选择的更多

select distinct score.student_id, student.sname, count(score.student_id) as Cnum
from score left join student on score.student_id=student.sid 
where score.course_id in (select course_id from score where score.student_id=1) and score.student_id !=1
group by score.student_id having Cnum=(select count(1) from score where score.student_id=1);

### 14. 查询和“002”号的同学学习的课程完全相同的其他同学学号和姓名；

- 个数相同
- 002学过的也学过

### 15. 删除学习“李平”老师课的score表记录；

### 16. 向score表中插入一些记录，这些记录要求符合以下条件：
- 没有上过编号“002”课程的同学学号；
- 插入“002”号课程的平均成绩；

思路：
- 由于insert 支持 
                inset into tb1(xx,xx) select x1,x2 from tb2;
        所有，获取所有没上过002课的所有人，获取002的平均成绩


### 17. 按平均成绩从低到高 显示所有学生的“生物”、“物理”、“体育”三门的课程成绩，按如下形式显示： 
        学生ID,生物,物理,体育,有效课程数,有效平均分；

下面是利用查找同一张表，内部的表的student_id和外部的表的student_id相同，从而在表中筛选出了匹配的值。

### 18. 查询各科成绩最高和最低的分：以如下形式显示：课程ID，最高分，最低分；

### 19. 按各科平均成绩从低到高和及格率的百分数从高到低顺序；

思路：case when .. then

### 20. 课程平均分从高到低显示（现实任课老师）；

select score.course_id, avg(if(isnull(score.num),0,score.num)),teacher.tname 
from course
left join score on course.cid = score.course_id
left join teacher on course.teacher_id = teacher.tid
group by score.course_id;

group by会遇到问题，Mysql解决SELECT list is not in GROUP BY clause and contains nonaggregated column 问题

1. 查询mysql 相关mode
    select @@global.sql_mode;

2. 可以看到模式中包含了ONLY_FULL_GROUP_BY，只要没有这个配置即可。
我的Mysql版本是5.7.21，默认是带了ONLY_FULL_GROUP_BY模式。

 ONLY_FULL_GROUP_BY,STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,
 ERROR_FOR_DIVISION_BY_ZERO,NO_AUTO_CREATE_USER,NO_ENGINE_SUBSTITUTION 

【重设模式值】

set  @@global.sql_mode=`STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_AUTO_CREATE_USER,NO_ENGINE_SUBSTITUTION`;

3.重启Mysql即可【mac重启mysql服务遇到问题】

### 21. 查询各科成绩前三名的记录:(不考虑成绩并列情况)

### 22. 查询每门课程被选修的学生数；

### 23. 查询出只选修了一门课程的全部学生的学号和姓名；

### 24. 查询男生、女生的人数；

### 25. 查询姓“张”的学生名单；

### 26. 查询同名同姓学生名单，并统计同名人数；

### 27. 查询每门课程的平均成绩，结果按平均成绩升序排列，平均成绩相同时，按课程号降序排列；

### 28. 查询平均成绩大于85的所有学生的学号、姓名和平均成绩；

select score.student_id,student.sname,avg(if(isnull(score.num),0,score.num)) as avg 
from score left join student on score.student_id=student.sid
group by score.student_id having avg >85;

### 29. 查询课程名称为“数学”，且分数低于60的学生姓名和分数；

### 30. 查询课程编号为003且课程成绩在80分以上的学生的学号和姓名；

### 31. 求选了课程的学生人数

select count(distinct student_id) from score;

### 32. 查询选修“李平老师”所授课程的学生中，成绩最高的学生姓名及其成绩；

### 33. 查询各个课程及相应的选修人数；

### 34. 查询不同课程但成绩相同的学生的学号、课程号、学生成绩；

### 35. 查询每门课程成绩最好的前两名；

select score.sid,score.course_id,score.num,T.first_num,T.second_num from score left join
    (
    select
        sid,
        (select num from score as s2 where s2.course_id = s1.course_id order by num desc limit 0,1) as first_num,
        (select num from score as s2 where s2.course_id = s1.course_id order by num desc limit 1,1) as second_num
    from
        score as s1
    ) as T
    on score.sid =T.sid
    where score.num <= T.first_num and score.num >= T.second_num

### 36. 检索至少选修两门课程的学生学号；

### 37. 查询全部学生都选修的课程的课程号和课程名；

### 38. 查询没学过“李平”老师讲授的任一门课程的学生姓名；

### 39. 查询两门以上不及格课程的同学的学号及其平均成绩；

### 40. 检索“004”课程分数小于60，按分数降序排列的同学学号；

### 41. 删除“002”同学的“001”课程的成绩；